# Load Data from Twitter API

connect to twitter via the tweepy library and with personal API keys

In [0]:
!pip install tweepy

In [0]:
# Large part of the code reused from 'https://github.com/stamatelou/twitter_sentiment_analysis/blob/master/twitter_connection.py'

# load libraries
from datetime import datetime
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json


In [0]:
# My API credentials

consumer_key = 'XP9WEtNESINs3LfjkyFYLIN8l'
consumer_secret = 'VeVcYCxyGNJHAq5mwzJZXiAfioXeAajhrcQBYcU1uDjPShqmCS'
access_token = '1126001584830390272-JnhR5hrvd0YX9tnIpBTj3uNkhx3tsW'
access_secret = 'yLXT7Tcsr0Xx7T7ldUfyHs8Eltrro18gVWnBm0Xo2Q54m'


In [0]:
# Create the stream listening function for extended tweets

class TweetsListener(StreamListener):
  # tweet object listens for the tweets
  def __init__(self, csocket):
    self.client_socket = csocket
    
  def on_data(self, data):
    try:  
      msg = json.loads( data )
#      print(f"new message: {datetime.now().isoformat(timespec='seconds')}")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text'])\
            .encode('utf-8'))         
#        print(msg['extended_tweet']['full_text'])
        print(f"new extended message: {datetime.now().isoformat(timespec='seconds')}")
      else:
        self.client_socket\
            .send(str(msg['text'])\
            .encode('utf-8'))
#        print(msg['text'])
        print(f"new message: {datetime.now().isoformat(timespec='seconds')}")
      return True
    except BaseException as e:
        print(f"Error on_data: {datetime.now().isoformat(timespec='seconds')}" + "%s" % str(e))
    return True
  
  def on_error(self, status):
    print(status)
    return True
  

In [0]:
# Define streaming authentication and message selection

def sendData(c_socket, keyword):
  print(f"start sending data from Twitter to socket: {datetime.now().isoformat(timespec='seconds')}")
  # authentication based on the credentials
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  
  # start sending data from the Streaming API 
  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track = keyword, languages=["en"])
  

In [0]:
# Start the Stream

s = socket.socket()
host = 'localhost'    
port = 9997
s.bind((host, port))
print(f"socket is ready: {datetime.now().isoformat(timespec='seconds')}")

# server (local machine) listens for connections
s.listen()
print(f"socket is listening: {datetime.now().isoformat(timespec='seconds')}")

# return the socket and the address on the other side of the connection (consumer side)
c_socket, addr = s.accept()
print(f"Received request at: {datetime.now().isoformat(timespec='seconds')}" + "from: " + str(addr))

# select here the keyword for the tweet data
sendData(c_socket, keyword = ['Bitcoin', 'bitcoin', 'Elon', 'elon', 'Musk', 'musk'])


--------------------------------------------------------------------------- 
 OSError Traceback (most recent call last)
 <command-2959400940596455> in <module> 
 4 host = 'localhost' 
 5 port = 9997 
 ----> 6 s . bind ( ( host , port ) ) 
 7 print ( f"socket is ready: {datetime.now().isoformat(timespec='seconds')}" ) 
 8 

 OSError : [Errno 98] Address already in use

In [0]:
s.close()
print(f"socket closed: {datetime.now().isoformat(timespec='seconds')}")

socket closed: 2021-05-16T07:33:51